### Setup

In [1]:
# import packages
import arcpy
import os
# set workspace
arcpy.env.workspace = "F:\GIS\PROJECTS\Transportation\Equity\Equity.gdb"
workspace           ="F:\GIS\PROJECTS\Transportation\Equity\Equity.gdb"
# set enviornment vairables
# set overwrite to true
arcpy.env.overwriteOutput = True
# in memory output file path
memory_workspace = "memory" + "\\"

# network dataset info
nd_path =  r"F:\GIS\GIS_DATA\Transportation\Basemap Features\Roads\Streets Network Dataset\Streets_NEW_ND.gdb\Streets_SDC_ND\Streets_CA_NV_ND"
nd_layer_name = "Streets_CA_NV"

# set local variables
# census data
dec2010  = "Tahoe_Census_2010"
dec2020  = "Tahoe_Census_2020"
acs2020  = "Tahoe_ACS_2020"
zvh      = 'ZVH_2020_ACS'
seniors  = 'Age_2020ACS'
disabled = 'HH_Disability_2020ACS'
poverty  = 'Poverty_2020ACS'
indisable= 'Individual_Disability_2020ACS'

# locations data
grocery  = "FreshFoodLocations"
medicare = "HealthCareLocations"
freebeach= 'Free_Beaches'
beaches  = "Public_Beaches"
meetings = "PublicMeetingLocations"

# slope classes
slope    = "C:\GIS\DB_CONNECT\Raster.sde\sde_imagery.SDE.Slope_Reclassified"
zonalstat_slopeClass = "ZonalStat_Hex_SlopeClassTable"

# transportation network data
streets    = "F:\GIS\PROJECTS\Transportation\Equity\Vector.sde\sde.SDE.Transportation\sde.SDE.Streets"
urban      = "F:\GIS\PROJECTS\Transportation\Equity\Vector.sde\sde.SDE.Jurisdictions\sde.SDE.UrbanAreas"

# service areas
freebeachSA= os.path.join(workspace,"Free_Beach_ServiceArea")
beachSA    = os.path.join(workspace,"Public_Beach_ServiceArea")
grocerySA  = os.path.join(workspace,"Fresh_Food_ServiceArea")
medicareSA = os.path.join(workspace,"Medicare_ServiceArea")
meetingSA  = os.path.join(workspace,"Public_Meetings_ServiceArea")

# hex bins
tahoeHex1km   = "TahoeTessellation_1sqkm"
tahoeHex01km  = "TahoeTessellation_0_1sqkm"
# 
hexZVH      = memory_workspace + "Hex_ZVH"
hexSenior   = memory_workspace + "Hex_SENIOR"
hexDisabled = memory_workspace + "Hex_DISABLED"
hexPoverty  = memory_workspace + "Hex_POVERTY"
# 
hexBeachSA    = memory_workspace + "Hex_BeachSA"
hexGrocerySA  = memory_workspace + "Hex_GrocerySA"
hexMedicareSA = memory_workspace + "Hex_MedicareSA"
hexMeetingSA  = memory_workspace + "Hex_MeetingSA"

In [7]:
# functions
# transfer attributes from spatial join feature class to tessellation
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
#     log.info("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
    # Use list comprehension to build a dictionary from arcpy SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList)}  
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))

# create service area
def createServiceArea(nd_path, nd_layer_name):
    # Check out the Network Analyst extension license.
    arcpy.CheckOutExtension("network")

    # Create a network dataset layer. The layer will be referenced using its name.
    arcpy.nax.MakeNetworkDatasetLayer(nd_path, nd_layer_name)

    # Instantiate a ServiceArea analysis object.
    service_area = arcpy.nax.ServiceArea(nd_layer_name)

    # Get the desired travel mode for the analysis.
    nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
    travel_mode = nd_travel_modes["New Travel Mode"]

    # Set properties.
    service_area.distanceUnits     = arcpy.nax.DistanceUnits.Miles
    service_area.defaultImpedanceCutoffs = [402.336,804.672,1609.344,3218.688, 6400]
    service_area.travelMode        = travel_mode
    service_area.allowAutoRelocate = True
    service_area.outputType        = arcpy.nax.ServiceAreaOutputType.Polygons
    service_area.geometryAtOverlap = arcpy.nax.ServiceAreaOverlapGeometry.Split
    service_area.searchTolerance   = 5000
    return service_area

# solve service area
def solveServiceArea(input_facilities, output_polygons, service_area):
    arcpy.management.Delete(output_polygons)
#     createServiceArea(nd_path,nd_layer_name)
    # load facilities
    service_area.load(arcpy.nax.ServiceAreaInputDataType.Facilities, 
                  input_facilities)
    
    # Solve the analysis.
    result = service_area.solve()
    # Export the results to a feature class. If the analysis failed print all the 
    # messages.
    if result.solveSucceeded:
        result.export(arcpy.nax.ServiceAreaOutputDataType.Polygons, output_polygons)
        print (f"Service Area generated for {input_facilities} and saved as {output_polygons}")
    else:
        arcpy.AddError("Analysis failed")
        # Print all the warning messages.
        for message in result.solverMessages(arcpy.nax.MessageSeverity.Warning):
            arcpy.AddWarning(message[-1])
        # Print all the error messages.
        for message in result.solverMessages(arcpy.nax.MessageSeverity.Error):
            arcpy.AddError(message[-1])


## Analysis

### Generate Tessellations

In [6]:
arcpy.management.GenerateTessellation(
    Output_Feature_Class="TahoeTessellation_0_1sqkm",
    Extent='-13388021.2623077 4678916.64488463 -13342410.4544964 4769413.84969433 PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
    Shape_Type="HEXAGON",
    Size="0.1 SquareKilometers",
    Spatial_Reference='PROJCS["NAD_1983_UTM_Zone_10N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5120900 -9998100 10000.0016462827;-100000 10000;-100000 10000;1.00000016391277E-03;0.001;0.001;IsHighPrecision',
    H3_Resolution=7
)

arcpy.management.SelectLayerByLocation(
    in_layer="TahoeTessellation_0_1sqkm",
    overlap_type="INTERSECT",
    select_features="TRPABoundary_Erase",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="INVERT"
)

arcpy.management.DeleteRows(
    in_rows="TahoeTessellation_0_1sqkm"
)

<Result 'TahoeTessellation_0_1sqkm'>

In [13]:
arcpy.management.GenerateTessellation(
    Output_Feature_Class="TahoeTessellation_1sqkm",
    Extent="737668.911313948 4288193.03147232 770952.921734459 4357302.88519489",
    Shape_Type="TRANSVERSE_HEXAGON",
    Size="1 SquareKilometers",
    Spatial_Reference='PROJCS["NAD_1983_UTM_Zone_10N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5120900 -9998100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision'
)

arcpy.management.SelectLayerByLocation(
    in_layer="TahoeTessellation_1sqkm",
    overlap_type="INTERSECT",
    select_features="TRPABoundary_Erase",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="INVERT"
)

arcpy.management.DeleteRows(
    in_rows= "TahoeTessellation_1sqkm"
)


<Result 'TahoeTessellation_1sqkm'>

### Add Fields

In [7]:
# Add fields to accept join values
 
# Set local variables
# inFeatures = "TahoeTessellation_1sqkm"
inFeatures = tahoeHex01km
fieldPrecision = 9

fieldName1  = "DISABILITY_DENSITY"
fieldAlias1 = "Disability Density"

fieldName2  = "POVERTY_DENSITY"
fieldAlias2 = "Poverty Density"

fieldName3  = "ZEROVHH_DENSITY"
fieldAlias3 = "Zero-Vehicle Household Density"

fieldName4  = "SENIOR_DENSITY"
fieldAlias4 = "Senior Density"

fieldName5  = "FOOD_ACCESS_DISTANCE"
fieldAlias5  = "Distance to Food Access"

fieldName6  = "BEACH_ACCESS_DISTANCE"
fieldAlias6 = "Distance to Free Public Beach"

fieldName7  = "MEDICARE_ACCESS_DISTANCE"
fieldAlias7 = "Distance to Medicare Facility"

fieldName8  = "MEETING_ACCESS_DISTANCE"
fieldAlias8 = "Distance to Public Meeting Location"

### Density Fields
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName1, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias1, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName2, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias2, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName3, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias3, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName4, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias4, 
                          field_is_nullable="NULLABLE")
### Distance From Fields
# Run AddField
arcpy.management.AddField(inFeatures, 
                          fieldName5, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias5, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName6, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias6, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName7, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias7, 
                          field_is_nullable="NULLABLE")
# Run AddField 
arcpy.management.AddField(inFeatures, 
                          fieldName8, "DOUBLE", 
                          fieldPrecision,
                          field_alias=fieldAlias8, 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          "MAJORITY_SLOPE_CLASS", "SHORT", 
                          field_alias="Majority Slope Class")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          "IS_URBAN", "TEXT", 
                          field_alias="Within Urban Area Boundary?", 
                          field_is_nullable="NULLABLE")

# Run AddField 
arcpy.management.AddField(inFeatures, 
                          "HAS_ROAD", "TEXT", 
                          field_alias="Road within Hex?", 
                          field_is_nullable="NULLABLE")


<Result 'F:\\GIS\\PROJECTS\\Transportation\\Equity\\Equity.gdb\\TahoeTessellation_0_1sqkm'>

### Census Data Processing

In [ ]:
# publish block group with attributes for 2020 density map....

### Spatial Joins & Zonal Stats

In [22]:
# zonal stats as table - hex bin & slope class
arcpy.ia.ZonalStatisticsAsTable(
    in_zone_data=tahoeHex01km ,
    zone_field="GRID_ID",
    in_value_raster=slope,
    out_table=zonalstat_slopeClass,
    ignore_nodata="DATA",
    statistics_type="MAJORITY"
)

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','MAJORITY_SLOPE_CLASS'], 
              zonalstat_slopeClass,  ['GRID_ID','MAJORITY'])
### Zero Vehicle Household Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, zvh, hexZVH, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','ZEROVHH_DENSITY'], 
              hexZVH,  ['GRID_ID','ZVH_DENS'])

### Seniors Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, seniors, hexSenior, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','SENIOR_DENSITY'], 
              hexSenior,  ['GRID_ID','OVER65_DENS'])

### Disabled Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, disabled, hexDisabled, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','DISABILITY_DENSITY'], 
              hexDisabled,  ['GRID_ID','HH_DISABILITY_DENS'])

### Poverty Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, poverty, hexPoverty, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','POVERTY_DENSITY'], 
              hexPoverty,  ['GRID_ID','HH_POVERTY_DENS'])

### Beach Access Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, freebeachSA, hexBeachSA, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','BEACH_ACCESS_DISTANCE'], 
              hexBeachSA,  ['GRID_ID','ToBreak'])

### Grocery Access Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, grocerySA, hexGrocerySA, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','FOOD_ACCESS_DISTANCE'], 
              hexGrocerySA,  ['GRID_ID','ToBreak'])

### Medicare Access Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, medicareSA, hexMedicareSA, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','MEDICARE_ACCESS_DISTANCE'], 
              hexMedicareSA,  ['GRID_ID','ToBreak'])

### Public Meeting Access Attribute Update -----------------------------------------------------------------------------------###
# Spatial Join
arcpy.SpatialJoin_analysis(tahoeHex01km, meetingSA, hexMeetingSA, 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", 
                           "HAVE_THEIR_CENTER_IN", "", "")

# transfer attributes to hex Layer
fieldJoinCalc(tahoeHex01km, ['GRID_ID','MEETING_ACCESS_DISTANCE'], 
              hexMeetingSA,  ['GRID_ID','ToBreak'])

### Urban Hex Attribute Update -----------------------------------------------------------------------------###

# Select parcels that intersect urban area boundary
urbanSelect = arcpy.SelectLayerByLocation_management(tahoeHex01km, 
                                                          'INTERSECT', 
                                                           urban, 
                                                           0, 
                                                          'NEW_SELECTION')
# Update field 1= yes 0 = no
with arcpy.da.UpdateCursor(urbanSelect, ['IS_URBAN']) as cursor:
    for row in cursor:
        row[0] = '1'
        cursor.updateRow(row) 
del cursor 

# switch the selection
urbanAreaSelect = arcpy.SelectLayerByAttribute_management(urbanSelect,'SWITCH_SELECTION')

# update other parcels
with arcpy.da.UpdateCursor(urbanAreaSelect, ['IS_URBAN']) as cursor:
    for row in cursor:
        row[0] = '0'
        cursor.updateRow(row)
del cursor

### Streets Hex Attribute Update -----------------------------------------------------------------------------###

# Select parcels that intersect urban area boundary
streetSelect = arcpy.SelectLayerByLocation_management(tahoeHex01km, 
                                                          'INTERSECT', 
                                                           streets, 
                                                           0, 
                                                          'NEW_SELECTION')
# Update field 1= yes 0 = no
with arcpy.da.UpdateCursor(streetSelect, ['HAS_ROAD']) as cursor:
    for row in cursor:
        row[0] = '1'
        cursor.updateRow(row) 
del cursor 

# switch the selection
streetNotSelect = arcpy.SelectLayerByAttribute_management(streetSelect,'SWITCH_SELECTION')

# update other parcels
with arcpy.da.UpdateCursor(streetNotSelect, ['HAS_ROAD']) as cursor:
    for row in cursor:
        row[0] = '0'
        cursor.updateRow(row)
del cursor

# clear selection
arcpy.SelectLayerByAttribute_management(tahoeHex01km, "CLEAR_SELECTION")

Started data transfer: 2023-07-07 14:52:25
Finished data transfer: 2023-07-07 14:52:27
Started data transfer: 2023-07-07 14:52:38
Finished data transfer: 2023-07-07 14:52:39
Started data transfer: 2023-07-07 14:52:50
Finished data transfer: 2023-07-07 14:52:51
Started data transfer: 2023-07-07 14:53:01
Finished data transfer: 2023-07-07 14:53:02
Started data transfer: 2023-07-07 14:53:12
Finished data transfer: 2023-07-07 14:53:13
Started data transfer: 2023-07-07 14:53:22
Finished data transfer: 2023-07-07 14:53:24
Started data transfer: 2023-07-07 14:53:33
Finished data transfer: 2023-07-07 14:53:34
Started data transfer: 2023-07-07 14:53:44
Finished data transfer: 2023-07-07 14:53:46
Started data transfer: 2023-07-07 14:53:56
Finished data transfer: 2023-07-07 14:53:57


<Result 'TahoeTessellation_0_1sqkm_La3'>

### Transform Values

In [23]:
# If the value is null, replace it with 0
with arcpy.da.UpdateCursor(tahoeHex01km, ['ZEROVHH_DENSITY', 'SENIOR_DENSITY', 
                                          'POVERTY_DENSITY', 'DISABILITY_DENSITY']) as cursor:
    for row in cursor:
        if row[0] is None:
            row[0] = 0
            cursor.updateRow(row)
        if row[1] is None:
            row[1] = 0
            cursor.updateRow(row)
        if row[2] is None:
            row[2] = 0
            cursor.updateRow(row)
        if row[3] is None:
            row[3] = 0
            cursor.updateRow(row)
del cursor

In [24]:
with arcpy.da.UpdateCursor(tahoeHex01km, ["BEACH_ACCESS_DISTANCE"]) as cursor:
    for row in cursor:
        if row[0]  == 402.336:
            row[0]  = 0.25
        elif row[0]  == 804.672:
            row[0]  = 0.5
        elif row[0]  == 1609.344:
            row[0]  = 1
        elif row[0] is None:
            row[0] = 3
        elif row[0]  > 1609.344 and row[0] <= 3218.688:
            row[0]  = 2
        elif row[0] > 3218.688 and row[0] <= 6400:
            row[0]= 3
        else:
            row[0] = 3

        cursor.updateRow(row)
del cursor

with arcpy.da.UpdateCursor(tahoeHex01km, ["FOOD_ACCESS_DISTANCE"]) as cursor:
    for row in cursor:
        if row[0]  == 402.336:
            row[0]  = 0.25
        elif row[0]  == 804.672:
            row[0]  = 0.5
        elif row[0]  == 1609.344:
            row[0]  = 1
        elif row[0] is None:
            row[0] = 3
        elif row[0]  > 1609.344 and row[0] <= 3218.688:
            row[0]  = 2
        elif row[0] > 3218.688 and row[0] <= 6400:
            row[0]= 3
        else:
            row[0] = 3

        cursor.updateRow(row)
del cursor

with arcpy.da.UpdateCursor(tahoeHex01km, ["MEDICARE_ACCESS_DISTANCE"]) as cursor:
    for row in cursor:
        if row[0]  == 402.336:
            row[0]  = 0.25
        elif row[0]  == 804.672:
            row[0]  = 0.5
        elif row[0]  == 1609.344:
            row[0]  = 1
        elif row[0] is None:
            row[0] = 3
        elif row[0]  > 1609.344 and row[0] <= 3218.688:
            row[0]  = 2
        elif row[0] > 3218.688 and row[0] <= 6400:
            row[0]= 3
        else:
            row[0] = 3

        cursor.updateRow(row)
del cursor

with arcpy.da.UpdateCursor(tahoeHex01km, ["MEETING_ACCESS_DISTANCE"]) as cursor:
    for row in cursor:
        if row[0]  == 402.336:
            row[0]  = 0.25
        elif row[0]  == 804.672:
            row[0]  = 0.5
        elif row[0]  == 1609.344:
            row[0]  = 1
        elif row[0] is None:
            row[0] = 3
        elif row[0]  > 1609.344 and row[0] <= 3218.688:
            row[0]  = 2
        elif row[0] > 3218.688 and row[0] <= 6400:
            row[0]= 3
        else:
            row[0] = 3

        cursor.updateRow(row)
del cursor

# with arcpy.da.UpdateCursor(tahoeHex01km, ["MAJORITY_SLOPE_CLASS"]) as cursor:
#     for row in cursor:
#         if row[0] == "1":
#             row[0]  = "0-5%"
#         elif row[0]  == "2":
#             row[0]  = "5-15%"
#         elif row[0]  == "3":
#             row[0]  = "15-30%"
#         elif row[0]  == "4":
#             row[0]  = "30-50%"
#         elif row[0]  == "5":
#             row[0]  = ">50%"
#         cursor.updateRow(row)
# del cursor

### Network Analysis

#### Create Service Area

In [9]:
createServiceArea(nd_path, nd_layer_name)

#### Solve Service Area

In [20]:
# Check out the Network Analyst extension license.
arcpy.CheckOutExtension("network")

# Create a network dataset layer. The layer will be referenced using its name.
arcpy.nax.MakeNetworkDatasetLayer(nd_path, nd_layer_name)

# Instantiate a ServiceArea analysis object.
service_area = arcpy.nax.ServiceArea(nd_layer_name)

# Get the desired travel mode for the analysis.
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["New Travel Mode"]

# Set properties.
service_area.distanceUnits     = arcpy.nax.DistanceUnits.Miles
service_area.defaultImpedanceCutoffs = [402.336,804.672,1609.344,3218.688, 6400]
service_area.travelMode        = travel_mode
service_area.allowAutoRelocate = True
service_area.outputType        = arcpy.nax.ServiceAreaOutputType.Polygons
service_area.geometryAtOverlap = arcpy.nax.ServiceAreaOverlapGeometry.Split
service_area.searchTolerance   = 5000

In [21]:

# solveServiceArea(freebeach,freebeachSA, service_area)
# solveServiceArea(grocery,grocerySA, service_area)
solveServiceArea(meetings,meetingSA, service_area)
# solveServiceArea(medicare,medicareSA, service_area)

Service Area generated for PublicMeetingLocations and saved as F:\GIS\PROJECTS\Transportation\Equity\Equity.gdb\Public_Meetings_ServiceArea


### Space Time Cube 